In [ ]:
import torch
import numpy as np
import math
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
import torch.nn as nn
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.metrics import r2_score
import random
import matplotlib as mpl
import os
import gc
from torch.utils.tensorboard import SummaryWriter
from datetime import date
mpl.rcParams['figure.dpi'] = 180

In [ ]:
# hyper-parameters
# batch_size = 11
device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
# LSTM Model
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, num_layers, seq_len,num_classes=1):
        super(RNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size1 = hidden_size1
        self.hidden_size2 = hidden_size2
        self.seq_len = seq_len

        self.bnn1 = nn.Linear(input_size, 32)
        self.bnn2 = nn.Linear(32,64)
        self.bnn3 = nn.Linear(64,64)
        self.bnn4 = nn.Linear(64,64)

        self.lstm1 = nn.LSTM(64, hidden_size1, num_layers, batch_first=True, bidirectional=True, dropout=0.5)
        self.bn_lstm1 = nn.BatchNorm1d(2*hidden_size1,device=device)  
        # self.lstm2 = nn.LSTM(2*hidden_size1, hidden_size2, num_layers, batch_first=True, bidirectional=True, dropout=0.5)
        # self.bn_lstm2 = nn.BatchNorm1d(2*hidden_size2,device=device)
        self.nn1 = nn.Linear(2*hidden_size1, 2*hidden_size1)
        self.nn2 = nn.Linear(2*hidden_size1, 512)
        self.nn3 = nn.Linear(512, 512)
        self.nn4 = nn.Linear(512, 256)
        self.nn5 = nn.Linear(256, 256)
        self.nn6 = nn.Linear(256, 128)
        self.nn7 = nn.Linear(128, 32)
        self.nn8 = nn.Linear(32, 1)

    
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()
        # self.batch = nn.BatchNorm1d()
        self.drop = nn.Dropout(p=0.5)


        
    def forward(self, x, array_lengths):
        # Set initial hidden states (and cell states for LSTM)
        # print(x.size(0))
        inital_seq_len = x.size(1)
        x = Variable(x.float()).to(device)

        x = torch.reshape(x, (x.size(0)*x.size(1), x.size(2)))

        ## before nn
        out = self.bnn1(x)
        out = self.relu(out)
        out = self.bnn2(out)
        out = self.relu(out)
        out = self.bnn3(out)
        out = self.relu(out)
        out = self.bnn4(out)
        out = self.relu(out)

        ## reshaping again
        out = torch.reshape(out, (-1, inital_seq_len, out.size(1)))
        # print(out.size())
        # print(aaaaa)

        # out = torch.permute(out, (0,2,1))
        
        pack = nn.utils.rnn.pack_padded_sequence(out, array_lengths, batch_first=True, enforce_sorted=False)
        h0 = Variable(torch.zeros(2*self.num_layers, x.size(0), self.hidden_size1).to(device))
        c0 = Variable(torch.zeros(2*self.num_layers, x.size(0), self.hidden_size1).to(device))
        h1 = Variable(torch.zeros(2*self.num_layers, self.hidden_size1, self.hidden_size2).to(device))
        c1 = Variable(torch.zeros(2*self.num_layers, self.hidden_size1, self.hidden_size2).to(device))
        
        # Forward propagate RNN
        out, _ = self.lstm1(pack, (h0,c0))
        del(h0)
        del(c0)
        # out, _ = self.lstm2(out, (h1,c1))
        gc.collect()
        unpacked, unpacked_len = torch.nn.utils.rnn.pad_packed_sequence(out, batch_first=True)
        this_batch_len = unpacked.size(1)
        out = unpacked
        # print('before', out.size())
        out = torch.reshape(out, (out.size(0)*out.size(1), out.size(2)))

        ##nn
        out = self.nn1(out)
        out = self.relu(out)
        out = self.nn2(out)
        out = self.relu(out)
        out = self.nn3(out)
        out = self.relu(out)
        out = self.nn4(out)
        out = self.relu(out)
        out = self.nn5(out)
        out = self.relu(out)
        out = self.nn6(out)
        out = self.relu(out)
        out = self.nn7(out)
        out = self.relu(out)
        out = self.nn8(out)
        
        ## reshaping
        out = torch.reshape(out, (-1, this_batch_len, 1))
        # print(out.size()) 
        # print(aaaaa)   
        

        return out

In [ ]:
epoch_check =177
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device is',device)

model_test = torch.load('/home/apa2237/Protein_BetaFactor/Model_35_primary/epoch_'+ str(epoch_check) + '.pth', map_location='cuda')
model_test.eval().to(device)

In [ ]:
x_test = np.load('x_test_35_pri.npy', allow_pickle=True)
# dataset preparation
class BetaDataset(Dataset) :
    def __init__(self,x,y, n_samples) :
        # data loading
        self.x = x
        self.y = y 
        self.n_samples = n_samples
        
        
    def __getitem__(self,index) :
        return self.x[index], self.y[index]

    def __len__(self) :    
        return self.n_samples      

test_dataset = BetaDataset(x_test,x_test,np.shape(x_test)[0])

test_loader = DataLoader(dataset=test_dataset,
                          batch_size=128,
                          shuffle=False,
                          num_workers=1)


In [ ]:
from math import nan

all_x = np.load('/home/apa2237/Protein_BetaFactor/dataloader_betanormalized/x_61046' + '.npy', allow_pickle=True)
all_y = np.load('/home/apa2237/Protein_BetaFactor/dataloader_betanormalized/y_61046' + '.npy', allow_pickle=True)

# test_loader = test_loader[15:20,:]
p_beta = torch.zeros((1)).to(device)
a_beta = torch.zeros((1)).to(device)
collection_test = 0
avg_pearson_count = 0
avg_pearson = 0
count_all = 0
with torch.no_grad():
  for i, (parameters, no_req) in enumerate(test_loader):
    # parameters = parameters.to(device)
    input_x = torch.from_numpy(all_x[parameters,:,:]).to(device)
    output_y = torch.from_numpy(all_y[parameters,:,:]).to(device)
    input_x = torch.reshape(input_x, (input_x.size(0), input_x.size(2), input_x.size(3)))
    output_y = torch.reshape(output_y, (output_y.size(0), output_y.size(2), output_y.size(3)))
    
    # forward pass  
    array_lengths = input_x[:,0,28]
    # print('step 1', type(array_lengths))
    array_lengths = array_lengths.int()
    # print('step 2', type(array_lengths))
    array_lengths = array_lengths.tolist()
    # print('step 3', type(array_lengths))
    # print(array_lengths.shape)
    # print(aaaa)
    outputs = model_test(input_x[:,:,0:21], array_lengths)
    outputs = torch.reshape(outputs, (-1,int(max(array_lengths))) )
    output_y = torch.reshape(output_y[:,0:int(max(array_lengths)), 0], (-1,int(max(array_lengths))))

    outputs = outputs.T
    output_y = output_y.T

    for j in range(input_x.size()[0]):    
      prot_len = int(input_x[j,0,28].item())
      # print(prot_len)
      if prot_len != 0:
        a_beta = torch.cat((torch.flatten(output_y[0:prot_len,j].float()), a_beta.float()),0)
        p_beta = torch.cat((torch.flatten(outputs[0:prot_len,j].float()), p_beta.float()),0)
        
        if math.isnan(np.corrcoef(outputs[0:prot_len,j].cpu(), output_y[0:prot_len,j].cpu())[0,1]):
          print('nan')
        else:
          # print(np.corrcoef(outputs[0:prot_len,j].cpu(), output_y[0:prot_len,j].cpu())[0,1])
          avg_pearson =  (avg_pearson*avg_pearson_count + np.corrcoef(outputs[0:prot_len,j].cpu(), output_y[0:prot_len,j].cpu())[0,1])/(avg_pearson_count+1)
          avg_pearson_count += 1
          count_all = count_all + prot_len

    # print(torch.flatten(outputs.float()).size())

    # a_beta = torch.cat((torch.flatten(output_y.float()), a_beta.float()),0)
    # p_beta = torch.cat((torch.flatten(outputs.float()), p_beta.float()),0)

    
    del(input_x)
    del(output_y)
    gc.collect()

a_beta = a_beta[1:,]
p_beta = p_beta[1:,]

print('Total data collected', a_beta.size(0))
print('Total data predicted', p_beta.size(0))
# print('Total point should be',count_all)
print('Avgerage Pearson coefficient is', avg_pearson)

In [ ]:
# combined = np.zeros((1,a_ebya))
with torch.no_grad():
    plt.figure(1)
    # plt.rcParams.update({'font.size': 16})
    plt.scatter(a_beta.cpu(), p_beta.cpu(), s=0.5)
    plt.plot(a_beta.cpu(), a_beta.cpu(),'k')
    # plt.title(f'R2 score:{r2_score(a_beta.cpu(), p_beta.cpu())}')
    plt.title(f'Pearson Correlation Coefficient:{round(avg_pearson,2)}, R^2 is {round(r2_score(a_beta.cpu(), p_beta.cpu()),2)}')
    # plt.plot(p_beta[50000:100000].cpu())
    plt.xlabel('Actual Normalized Beta factor')
    plt.ylabel('Predicted Normalized Beta factor')
    # plt.xlim([0,80])
    # plt.ylim([0,80])
    plt.show()